In [1]:
import os 
import time
import json
from pathlib import Path
import pandas as pd
import torch.nn as nn
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch import optim

- design a neural network to predict 3d pose from 2d coords
- options:

1. simple MLP:
    - input: single frame with camera views concatenated -> (C, J, 2) 
    - output: 3d coords (J, 3)



In [2]:
NUM_JOINTS = 20
NUM_CAMS = 6
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
print(device)

cuda


In [4]:
# for metrics it needs to be (J, 3)

# for a test, run a sequence and network will output (60) length vector for each frame in the sequence
# (60) -> (20,3)


In [ ]:
# model
class SimpleMLP(nn.Module):

    def __init__(self, activation_function, dropout=0.5):
        super(SimpleMLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(in_features = NUM_JOINTS * NUM_CAMS * 3, out_features = 1024),
            nn.BatchNorm1d(1024),
            nn.Dropout(dropout),
            nn.Linear(in_features = 1024, out_features = 1024),
            nn.BatchNorm1d(1024),
            nn.Dropout(dropout),
            nn.Linear(in_features= 1024, out_features= NUM_JOINTS * 3)
        )


        self.activation_function = activation_function

    def forward(self, x: torch.Tensor):
        """Defines a forward pass through the network."""
        # protect the data and turn all nans to zeros
        x[torch.isnan(x)] = 0
        # preserve batch dimension, but flatten the rest (C, J, 2) to single vector
        x = x.view(x.size(0), -1) 

        return self.net(x)
        


In [6]:
sum(p.numel() for p in model.parameters())

NameError: name 'model' is not defined

In [7]:
# Evaluation function
def evaluate_model(model, criterion, loader: DataLoader, device: str):
    model.eval()  # Set the model to evaluation mode
    val_running_loss = 0.0

    with torch.no_grad():  # Disable gradient computation
        for dets_2d, gt_3d in loader:
            dets_2d = dets_2d.to(device)
            gt_3d = gt_3d.to(device)
            # flatten the targets before loss computation
            gt_3d = gt_3d.view(gt_3d.size(0), -1)

            preds = model(dets_2d)  # Forward pass
            loss = criterion(preds, gt_3d)  # Compute validation loss
            val_running_loss += loss.item()

    # Calculate validation metrics
    val_loss = val_running_loss / len(loader)
    return val_loss

In [8]:
# Training function
def train_model(model ,criterion, optimizer, train_loader, val_loader, device, epochs=10):
    """

    """
    for epoch in range(epochs):
        print(f"WE ARE IN EPOCH {epoch}")
        # Training phase
        model.train()  # Set the model to training mode
        running_loss = 0.0

        for dets_2d, gt_3d in train_loader:

            dets_2d = dets_2d.to(device)
            gt_3d = gt_3d.to(device)
            # flatten 3d gt to match model output
            gt_3d = gt_3d.view(gt_3d.size(0), -1)

            optimizer.zero_grad()  # Clear previous gradients


            preds = model(dets_2d)  # Forward pass
            loss = criterion(preds, gt_3d)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights


            running_loss += loss.item()  # Accumulate batch loss

        # Calculate epoch training accuracy
        # Validation phase (call separate function)
        val_loss = evaluate_model(model, criterion,val_loader,device) #CAUTION: See note above

        # Print progress
        print(
            f"Epoch [{epoch+1}/{epochs}], "
            f"Train Loss: {running_loss/len(train_loader):.4f}, "
            f"Val Loss: {val_loss:.4f}"
        )

    return val_loss

In [9]:
# load the data
from data_loader import get_data_loaders

train_loader, val_loader, test_loader = get_data_loaders()
model = SimpleMLP(nn.ReLU()).to(device)
optimizer = optim.AdamW(model.parameters())

train_model(model, nn.MSELoss(), optimizer, train_loader, val_loader, device)

WE ARE IN EPOCH 0


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x360 and 240x1024)